<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [8]:
using Pkg; Pkg.activate(".")
Pkg.instantiate()

  Activating project at `a:\07Dissertation\01 Seminare\AMO\Abgabe\Team_assignment_AMO`

┌ Warning: The active manifest file has dependencies that were resolved with a different julia version (1.10.0). Unexpected behavior may occur.
└ @ nothing A:\07Dissertation\01 Seminare\AMO\Abgabe\Team_assignment_AMO\Manifest.toml:0


   Installed CodecBzip2 ───────── v0.8.2
   Installed CodecZlib ────────── v0.7.4


   Installed TranscodingStreams ─ v0.10.3


   Installed JuMP ─────────────── v1.19.0


...

  ✓ CodecBzip2


  ✓ IJulia


  ✓ MathOptInterface


  ✓ HiGHS


  ✓ JuMP
  9 dependencies successfully precompiled in 140 seconds. 56 already precompiled.


In [9]:
using JuMP,HiGHS,DataFrames


In [61]:
#nP=1:3                                  #number of generation units
#nW=1:1                                  #number of stochastic generation units
#nt=1:1                                  #number of time intervalls 
#nS=1:2                                  #number of scenarios
#nL=1:2                                  #number of loads
#nB=1:1                                  #number of buses
#C=[100, 80, 50]                         #unit cost of generation
#C_RU=[20, 10, 15]                       #cost of up reserve commitment
#C_RD=[25, 15, 15]                       #cost of down reserve commitment
#C_U=[10, 15, 20]                        #cost of up reserve commitment
#C_D=[30, 10, 15]                        #cost of down reserve commitment
#pi=[0.7, 0.3]                           #probabilities of the scenarios
#V_LOL=[20, 25]                          #value of the lost load
#C_cur=[15]                              #cost of curtailment at turbine
#L= 90 .+ 30 * randn(2, 1)               #loads for every time intervall
#W_realized = 70 .+ 15 * randn(1, 2, 1)  #create random scenarios for four time points
#P_max=[70, 80, 150];                    #maximum production capacity

nP=1:3                                  #number of generation units
nW=1:1                                  #number of stochastic generation units
nt=1:1                                  #number of time intervalls 
nS=1:2                                  #number of scenarios
nL=1:2                                  #number of loads
nB=1:2                                  #number of buses
C=[10, 30, 35]                          #unit cost of generation
C_RU=[16, 13, 10]                       #cost of up reserve commitment
C_RD=[15, 12, 9]                        #cost of down reserve commitment
C_U=[10, 30, 35]                        #cost of up reserve commitment
C_D=[10, 30, 35]                        #cost of down reserve commitment
pi=[0.6, 0.4]                           #probabilities of the scenarios
V_LOL=[200, 200]                        #value of the lost load
C_cur=[0] #[15]                         #cost of curtailment at turbine
L= [40,100]               #loads for every time intervall
W_realized = [[[50] [10]] [[50] [10]] [[50] [10]]] #create random scenarios for four time points w,s,t
#[[[111 112] [121 122]] [[211 212] [221 222]] [[311 312] [321 322]]]
P_max=[50, 110, 100];                   #maximum production capacity
b = [[1, 1/0.13] [1/0.13, 1]]                     #loss of transmission
LC_Max = [[0,100] [100, 0]]          #maximum transmission quantity between two buses

In [62]:
data= Dict(
    :nP => nP,
    :nW => nW,
    :nt => nt,
    :nS => nS,
    :nL => nL,
    :C => C,
    :C_RU => C_RU,
    :C_RD => C_RD,
    :P_max => P_max,
    :C_U => C_U,
    :C_D => C_D,
    :pi => pi,
    :V_LOL => V_LOL,
    :C_cur => C_cur,
    :L => L,
    :W_realized => W_realized,
    :P_max => P_max,
    :b => b,
    :LC_Max => LC_Max
    );
print(W_realized[1,2,1])

10

In [63]:
function init_variables(model::JuMP.Model)
    P=@variable(model, P[p in data[:nP], t in data[:nt]] >=0)
    R_U=@variable(model, R_U[p in data[:nP], t in data[:nt]] >=0)
    R_D=@variable(model, R_D[p in data[:nP], t in data[:nt]] >=0)
    r_U=@variable(model, r_U[p in data[:nP], s in data[:nS], t in data[:nt]] >=0)
    r_D=@variable(model, r_D[p in data[:nP], s in data[:nS], t in data[:nt]] >=0)
    L_Shed=@variable(model, L_Shed[l in data[:nL], s in data[:nS], t in data[:nt]] >=0)
    W_spill=@variable(model, W_spill[w in data[:nW], s in data[:nS], t in data[:nt]] >=0)
    W_s=@variable(model, W_s[w in data[:nW], t in data[:nt]] >=0)
    Delta=@variable(model, Delta[i in data[:nB], j in data[:nB]])
    PF=@variable(model, PF[s in data[:nS], i in data[:nB], j in data[:nB]])
    
    vars= Dict(
        :P=> P,
        :R_U => R_U,
        :R_D => R_D,
        :r_U => r_U,
        :r_D => r_D,
        :L_Shed => L_Shed,
        :W_spill => W_spill,
        :W_s => W_s,
        :Delta => Delta
        :PF => PF
    )
    
    return vars
end;

In [64]:
function init_constraints(model::JuMP.Model, data::Dict, vars::Dict)
    @constraints(model, begin 
            c1[p in data[:nP], t in data[:nt]], vars[:P][p, t]+ vars[:R_U][p, t]<= P_max[p]
            c2[p in data[:nP] ,t in data[:nt]], vars[:P][p, t]- vars[:R_D][p, t]>= 0
            c3[p in data[:nP], t in data[:nt], s in data[:nS]], vars[:r_U][p, s, t] <= vars[:R_U][p,t]
            c4[p in data[:nP], t in data[:nt], s in data[:nS]], vars[:r_D][p, s, t] <= vars[:R_D][p,t]
            c5[s in data[:nS], b1 in data[:nB], b2 in data[:nB], vars[:PF[b1, b2]] <= data[:LC_Max[b1,b2]]]
            c6[s in data[:nS], b1 in data[:nB], b2 in data[:nB], vars[:Delta[b1, b2]] - vars[:Delta[b2, b1]] * data[:b[b1, b2]] == vars[:PF[b1, b2]]]
            Power_balance_day_ahead[t in data[:nt]], 
            sum(vars[:P][p,t] for p in data[:nP]) + sum(vars[:W_s][w, t] for w in data[:nW]) - sum(L[l, t] for l in data[:nL]) - vars[:PF[b1, b2]] == 0
            Power_balance_at_stage[s in data[:nS], t in data[:nt]], 
            sum(vars[:r_U][p, s, t] for p in data[:nP]) - sum(vars[:r_D][p, s, t] for p in data[:nP]) - 
            sum(vars[:L_Shed][l, s, t] for l in data[:nL]) + sum(data[:W_realized][w,s,t] -vars[:W_s][w, t] for w in data[:nW])== 0      
    end)
end;

In [65]:
function init_obj_function(model, data, vars)
    @objective(model, Min, sum(vars[:P][p,t]* C[p] + vars[:R_U][p,t] * C_RU[p] + vars[:R_D][p,t] * C_RD[p] for p in data[:nP], t in data[:nt]) +
    sum(pi[s] * (sum(data[:C_U][p] * vars[:r_U][p, s ,t] + data[:C_D][p] * vars[:r_D][p, s, t] for p in data[:nP], t in data[:nt]) +
                sum(data[:C_cur][w] * vars[:W_spill][w, s, t] for w in data[:nW], t in data[:nt]) 
                + sum(data[:V_LOL][l] * vars[:L_Shed][l, s, t] for l in data[:nL], t in data[:nt])) for s in data[:nS])
    )
end;

In [66]:
model=Model(HiGHS.Optimizer)
vars=init_variables(model)
init_constraints(model, data, vars)
init_obj_function(model, data, vars)
print(model)

Min 10 P[1,1] + 16 R_U[1,1] + 15 R_D[1,1] + 30 P[2,1] + 13 R_U[2,1] + 12 R_D[2,1] + 35 P[3,1] + 10 R_U[3,1] + 9 R_D[3,1] + 6 r_U[1,1,1] + 6 r_D[1,1,1] + 18 r_U[2,1,1] + 18 r_D[2,1,1] + 21 r_U[3,1,1] + 21 r_D[3,1,1] + 120 L_Shed[1,1,1] + 120 L_Shed[2,1,1] + 4 r_U[1,2,1] + 4 r_D[1,2,1] + 12 r_U[2,2,1] + 12 r_D[2,2,1] + 14 r_U[3,2,1] + 14 r_D[3,2,1] + 80 L_Shed[1,2,1] + 80 L_Shed[2,2,1]
Subject to
 Power_balance_day_ahead[1] : P[1,1] + P[2,1] + P[3,1] + W_s[1,1] == 140
 Power_balance_at_stage[1,1] : r_U[1,1,1] + r_U[2,1,1] + r_U[3,1,1] - r_D[1,1,1] - r_D[2,1,1] - r_D[3,1,1] - L_Shed[1,1,1] - L_Shed[2,1,1] - W_s[1,1] == -50
 Power_balance_at_stage[2,1] : r_U[1,2,1] + r_U[2,2,1] + r_U[3,2,1] - r_D[1,2,1] - r_D[2,2,1] - r_D[3,2,1] - L_Shed[1,2,1] - L_Shed[2,2,1] - W_s[1,1] == -10
 c2[1,1] : P[1,1] - R_D[1,1] >= 0
 c2[2,1] : P[2,1] - R_D[2,1] >= 0
 c2[3,1] : P[3,1] - R_D[3,1] >= 0
 c1[1,1] : P[1,1] + R_U[1,1] <= 50
 c1[2,1] : P[2,1] + R_U[2,1] <= 110
 c1[3,1] : P[3,1] + R_U[3,1] <= 100
 c3[1,

In [67]:
optimize!(model)

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
21 rows, 24 cols, 56 nonzeros
20 rows, 23 cols, 56 nonzeros
Presolve : Reductions: rows 20(-1); columns 23(-5); elements 56(-2)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -9.9999686469e+00 Ph1: 3(3); Du: 1(9.99997) 0s
          7     2.6600000000e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 7
Objective value     :  2.6600000000e+03
HiGHS run time      :          0.00


In [76]:
# Print the values of the variables
println("Variable values:")
println("P:")
for p in data[:nP], t in data[:nt]
    println("P[$p]: ", value(vars[:P][p, t]))
end

println("\nR_U:")
for p in data[:nP], t in data[:nt]
    println("R_U[$p]: ", value(vars[:R_U][p, t]))
end

println("\nR_D:")
for p in data[:nP], t in data[:nt]
    println("R_D[$p]: ", value(vars[:R_D][p, t]))
end

println("\nL_Shed:")
for l in data[:nL], s in data[:nS], t in data[:nt]
    println("L_Shed[$l, $s, $t]: ", value(vars[:L_Shed][l, s, t]))
end

println("\nW_spill:")
for w in data[:nW], s in data[:nS], t in data[:nt]
    println("W_spill[$w, $s, $t]: ", value(vars[:W_spill][w, s, t]))
end

println("\nW_realized:")
for w in data[:nW], s in data[:nS], t in data[:nt]
    println("W_realized[$w, $s, $t]: ", value(data[:W_realized][w, s, t]))
end

println("\nW_s:")
for w in data[:nW], t in data[:nt]
    println("W_s[$w, $t]: ", value(vars[:W_s][w, t]))
end

println("\nr_U:")
for p in data[:nP], s in data[:nS], t in data[:nt]
    println("r_U[$p, $s, $t]: ", value(vars[:r_U][p, s, t]))
end

println("\nr_D:")
for p in data[:nP], s in data[:nS], t in data[:nt]
    println("r_D[$p, $s, $t]: ", value(vars[:r_D][p, s, t]))
end

println("\nLoad")
for l in data[:nL]
    print("L[$l]:", data[:L][l])
end

print("\n\nsum:", sum(data[:L]))
"""
print("Day-ahead Energy Cost" & sum(value(vars[:P][p,t])* C[p] for p in data[:nP], t in data[:nt]))

print("Reserve capacity costs" & vars[:R_U][p,t] * C_RU[p] + vars[:R_D][p,t] * C_RD[p] for p in data[:nP], t in data[:nt])

print("Balancing costs in scenarios" & sum(pi[s] * (sum(data[:C_U][p] * vars[:r_U][p, s ,t] + data[:C_D][p] * vars[:r_D][p, s, t] for p in data[:nP], t in data[:nt]) +
            sum(data[:C_cur][w] * vars[:W_spill][w, s, t] for w in data[:nW], t in data[:nt]) 
            + sum(data[:V_LOL][l] * vars[:L_Shed][l, s, t] for l in data[:nL], t in data[:nt])) for s in data[:nS]))
"""

Variable values:
P:
P[1]: 50.0
P[2]: 40.0
P[3]: 0.0

R_U:
R_U[1]: 0.0
R_U[2]: 0.0
R_U[3]: 40.0

R_D:
R_D[1]: 0.0
R_D[2]: 0.0
R_D[3]: 0.0

L_Shed:
L_Shed[1, 1, 1]: 0.0
L_Shed[1, 2, 1]: 0.0
L_Shed[2, 1, 1]: 0.0
L_Shed[2, 2, 1]: 0.0

W_spill:


W_spill[1, 1, 1]: 0.0
W_spill[1, 2, 1]: 0.0

W_realized:
W_realized[1, 1, 1]: 50
W_realized[1, 2, 1]: 10

W_s:
W_s[1, 1]: 50.0

r_U:
r_U[1, 1, 1]: 0.0
r_U[1, 2, 1]: -0.0
r_U[2, 1, 1]: 0.0
r_U[2, 2, 1]: -0.0
r_U[3, 1, 1]: 0.0
r_U[3, 2, 1]: 40.0

r_D:
r_D[1, 1, 1]: 0.0
r_D[1, 2, 1]: 0.0
r_D[2, 1, 1]: 0.0
r_D[2, 2, 1]: 

0.0
r_D[3, 1, 1]: 0.0
r_D[3, 2, 1]: 0.0

Load
L[1]:40L[2]:100

sum:

140

MethodError: MethodError: no method matching &(::String, ::Float64)

Closest candidates are:
  &(::Any, ::Any, !Matched::Any, !Matched::Any...)
   @ Base operators.jl:578
